In [10]:
'''
Tensorflow 2.0
'''

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

In [ ]:
# Hyperparameter Tunning

num_epochs = 1
batch_size = 64

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28,28,1)
num_classes = 10

In [11]:
# 1. preprocess

(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

In [12]:
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

train_x = train_x / 255.
test_x = test_x/ 255.

In [14]:
# 2. Build Model

inputs = layers.Input(input_shape)
net = layers.Conv2D(32,(3,3),padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,(3,3),padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64,(3,3),padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,(3,3),padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs= inputs, outputs=net, name='Basic_CNN')

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function
              metrics=['accuracy']) # Metrics / Accuracy

In [17]:
# 3. Training

model.fit(train_x, train_y,
          batch_size=batch_size,
          shuffle=True)

model.evaluate(test_x, test_y, batch_size=batch_size)

Train on 60000 samples
10000/10000 [==============================] - 6s 566us/sample - loss: 0.0434 - accuracy: 0.9853


[0.04343047028146684, 0.9853]

In [2]:
'''
PyTorch
'''

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [3]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

In [4]:
# 1. Model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
# 2. Preporcess

torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs={'num_workers':1, 'pin_memory':True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=True, download=True,
                 transform=transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize((0.1307,),(0.3081,))
                 ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, transform=transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize((0.1307,),(0.3081,))
                 ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100.1%

Extracting ../dataset\MNIST\raw\train-images-idx3-ubyte.gz to ../dataset\MNIST\raw


113.5%

Extracting ../dataset\MNIST\raw\train-labels-idx1-ubyte.gz to ../dataset\MNIST\raw


100.4%

Extracting ../dataset\MNIST\raw\t10k-images-idx3-ubyte.gz to ../dataset\MNIST\raw


180.4%C:\Users\pmcsh\Anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ../dataset\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../dataset\MNIST\raw
Processing...
Done!


In [7]:
# 3. Optimization
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [8]:
# 4. Training
for epoch in range(1, epochs + 1):
    # train mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # 한 번 학습 완료
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)\tLoss: {:.6f}]'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100 * batch_idx/ len(train_loader), loss.item()
            ))
        # 표시를 위해서 윗부분 작성
        
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)	Loss: 2.300880]
Train Epoch: 1 [6400/60000 (11%)	Loss: 2.239080]
Train Epoch: 1 [12800/60000 (21%)	Loss: 2.136498]
Train Epoch: 1 [19200/60000 (32%)	Loss: 1.950311]
Train Epoch: 1 [25600/60000 (43%)	Loss: 1.701460]
Train Epoch: 1 [32000/60000 (53%)	Loss: 1.167189]
Train Epoch: 1 [38400/60000 (64%)	Loss: 0.817326]
Train Epoch: 1 [44800/60000 (75%)	Loss: 0.626718]
Train Epoch: 1 [51200/60000 (85%)	Loss: 0.475071]
Train Epoch: 1 [57600/60000 (96%)	Loss: 0.447336]

Test set: Average Loss: 0.4469, Accuracy: 8859/10000 (89%)

